In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"  
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [13]:
from implicit.als import AlternatingLeastSquares
from rectools.metrics import MAP
from rectools.models import ImplicitALSWrapperModel
from rectools import Columns
from rectools.dataset import Dataset
from rectools.tools import UserToItemAnnRecommender
from pathlib import Path
import pandas as pd
import nmslib

## Loading data

In [4]:
DATA_PATH = Path("../../data/03_primary")

In [5]:
users_features_full = pd.read_csv(DATA_PATH / 'au_featured_users_full.csv')
items_features_full = pd.read_csv(DATA_PATH / 'prepared_featured_items_full.csv')
interactions_full = pd.read_csv(DATA_PATH / 'au_interactions_full.csv')
items_descriptions = pd.read_csv(DATA_PATH / 'items_descriptions_cleared.csv')

In [6]:
full_dataset = Dataset.construct(
    interactions_df=interactions_full,
)

full_featured_dataset = Dataset.construct(
    interactions_df=interactions_full,
    user_features_df=users_features_full,
    cat_user_features=["sex", "age", "income"],
    item_features_df=items_features_full,
    cat_item_features=["genre", "content_type","age_rating","country","release_year_bin"],
)

In [7]:
# IDs from data preparation notebook
artificial_users_ids = [1097559, 1097561, 1097563]

## Models training

All hyperparameters was taken from ALS notebook

In [8]:
K_RECOS = 10
RANDOM_STATE = 42
NUM_THREADS = 4

In [9]:
als_without_features = ImplicitALSWrapperModel(
    model = AlternatingLeastSquares(
        factors = 2,
        regularization = 0.03716111608869566,
        iterations = 42,
        use_cg = True,
        use_gpu = False,
        num_threads = NUM_THREADS,
        random_state = RANDOM_STATE
    )
)

als_without_features.fit(full_dataset)

In [10]:
als_with_features = ImplicitALSWrapperModel(
    model = AlternatingLeastSquares(
        factors = 3,
        regularization = 0.0018347283256986877,
        iterations = 13,
        use_cg = True,
        use_gpu = False,
        num_threads = NUM_THREADS,
        random_state = RANDOM_STATE
    ),
    fit_features_together = True,
)

als_with_features.fit(full_featured_dataset)

In [11]:
user_embeddings, item_embeddings = als_with_features.get_vectors()

In [14]:
ann = UserToItemAnnRecommender(
    user_vectors=user_embeddings,
    item_vectors=item_embeddings,
    user_id_map=full_featured_dataset.user_id_map,
    item_id_map=full_featured_dataset.item_id_map,
    index_init_params={
        'method' : 'hnsw',
        'space' : 'negdotprod',
        'data_type' : nmslib.DataType.DENSE_VECTOR
    }
)

In [15]:
ann.fit()

## Checking recommendations

1097559 - Mother with kid

1097561 - Superheroics fan

1097563 - Born in USSR

### ALS without features

In [16]:
als_wo_features_reco = als_without_features.recommend(
    users=artificial_users_ids,
    dataset=full_dataset,
    k=K_RECOS,
    filter_viewed=True,
)

In [17]:
als_wo_features_reco = als_wo_features_reco.merge(items_descriptions, on='item_id')\
    .sort_values(by=['user_id','rank'])

#### Recos for mother with kid

In [18]:
als_wo_features_reco[:10]

,user_id,item_id,score,rank,content_type,title,release_year,genres,countries,age_rating
0,1097559,7626,0.205796,1,film,Мстители: Война бесконечности,2018.0,"боевики, фантастика, приключения",США,16.0
3,1097559,12173,0.202589,2,film,Мстители: Финал,2019.0,"боевики, драмы, фантастика",США,16.0
6,1097559,16166,0.192270,3,film,Зверополис,2016.0,"приключения, мультфильм, детективы, комедии",США,6.0
9,1097559,9728,0.187336,4,film,Гнев человеческий,2021.0,"боевики, триллеры","Великобритания, США",18.0
11,1097559,3182,0.187101,5,film,Ральф против Интернета,2018.0,"мультфильм, приключения, фантастика, семейное,...",США,6.0
14,1097559,11237,0.183676,6,film,День города,2021.0,комедии,Россия,16.0
17,1097559,14317,0.178742,7,film,Веном,2018.0,"популярное, фантастика, триллеры, боевики, ужасы",США,16.0
20,1097559,13018,0.167502,8,film,Король лев (2019),2019.0,"драмы, мультфильм, музыкальные",США,6.0
23,1097559,4457,0.167262,9,film,2067: Петля времени,2020.0,"драмы, фантастика",Австралия,16.0
26,1097559,4436,0.164073,10,film,Ford против Ferrari,2019.0,драмы,"США, Франция",16.0


#### Recos for superheroics fan

In [19]:
als_wo_features_reco[10:20]

,user_id,item_id,score,rank,content_type,title,release_year,genres,countries,age_rating
28,1097561,7571,0.186155,1,film,100% волк,2020.0,"мультфильм, приключения, семейное, фэнтези, ко...","Австралия, Бельгия",6.0
1,1097561,7626,0.173646,2,film,Мстители: Война бесконечности,2018.0,"боевики, фантастика, приключения",США,16.0
4,1097561,12173,0.170421,3,film,Мстители: Финал,2019.0,"боевики, драмы, фантастика",США,16.0
7,1097561,16166,0.161210,4,film,Зверополис,2016.0,"приключения, мультфильм, детективы, комедии",США,6.0
12,1097561,3182,0.157422,5,film,Ральф против Интернета,2018.0,"мультфильм, приключения, фантастика, семейное,...",США,6.0
18,1097561,14317,0.148785,6,film,Веном,2018.0,"популярное, фантастика, триллеры, боевики, ужасы",США,16.0
15,1097561,11237,0.141144,7,film,День города,2021.0,комедии,Россия,16.0
21,1097561,13018,0.138874,8,film,Король лев (2019),2019.0,"драмы, мультфильм, музыкальные",США,6.0
27,1097561,4436,0.132068,9,film,Ford против Ferrari,2019.0,драмы,"США, Франция",16.0
24,1097561,4457,0.129889,10,film,2067: Петля времени,2020.0,"драмы, фантастика",Австралия,16.0


#### Recos for old man

In [20]:
als_wo_features_reco[20:]

,user_id,item_id,score,rank,content_type,title,release_year,genres,countries,age_rating
29,1097563,7571,0.000431,1,film,100% волк,2020.0,"мультфильм, приключения, семейное, фэнтези, ко...","Австралия, Бельгия",6.0
2,1097563,7626,0.000383,2,film,Мстители: Война бесконечности,2018.0,"боевики, фантастика, приключения",США,16.0
5,1097563,12173,0.000377,3,film,Мстители: Финал,2019.0,"боевики, драмы, фантастика",США,16.0
8,1097563,16166,0.000357,4,film,Зверополис,2016.0,"приключения, мультфильм, детективы, комедии",США,6.0
13,1097563,3182,0.000348,5,film,Ральф против Интернета,2018.0,"мультфильм, приключения, фантастика, семейное,...",США,6.0
10,1097563,9728,0.000343,6,film,Гнев человеческий,2021.0,"боевики, триллеры","Великобритания, США",18.0
16,1097563,11237,0.000340,7,film,День города,2021.0,комедии,Россия,16.0
19,1097563,14317,0.000332,8,film,Веном,2018.0,"популярное, фантастика, триллеры, боевики, ужасы",США,16.0
22,1097563,13018,0.000311,9,film,Король лев (2019),2019.0,"драмы, мультфильм, музыкальные",США,6.0
25,1097563,4457,0.000310,10,film,2067: Петля времени,2020.0,"драмы, фантастика",Австралия,16.0


### ALS with features

In [21]:
als_with_features_reco = als_with_features.recommend(
    users=artificial_users_ids,
    dataset=full_featured_dataset,
    k=K_RECOS,
    filter_viewed=True,
)

In [22]:
als_with_features_reco = als_with_features_reco.merge(items_descriptions, on='item_id')\
    .sort_values(by=['user_id','rank'])

#### Recos for mother with kid

In [23]:
als_with_features_reco[:10]

,user_id,item_id,score,rank,content_type,title,release_year,genres,countries,age_rating
0,1097559,15297,0.450052,1,series,Клиника счастья,2021.0,"драмы, мелодрамы",Россия,18.0
2,1097559,9728,0.267371,2,film,Гнев человеческий,2021.0,"боевики, триллеры","Великобритания, США",18.0
5,1097559,4151,0.261628,3,series,Секреты семейной жизни,2021.0,комедии,Россия,18.0
6,1097559,10440,0.238132,4,series,Хрустальный,2021.0,"триллеры, детективы",Россия,18.0
8,1097559,3734,0.211915,5,film,Прабабушка легкого поведения,2021.0,комедии,Россия,16.0
11,1097559,6809,0.186515,6,film,Дуров,2021.0,документальное,Россия,16.0
13,1097559,13865,0.162869,7,film,Девятаев,2021.0,"драмы, военные, приключения",Россия,12.0
16,1097559,16166,0.098390,8,film,Зверополис,2016.0,"приключения, мультфильм, детективы, комедии",США,6.0
17,1097559,13018,0.097685,9,film,Король лев (2019),2019.0,"драмы, мультфильм, музыкальные",США,6.0
18,1097559,142,0.097565,10,film,Маша,2020.0,"драмы, триллеры",Россия,16.0


#### Recos for superheroins fan

In [24]:
als_with_features_reco[10:20]

,user_id,item_id,score,rank,content_type,title,release_year,genres,countries,age_rating
3,1097561,9728,0.280038,1,film,Гнев человеческий,2021.0,"боевики, триллеры","Великобритания, США",18.0
14,1097561,13865,0.253946,2,film,Девятаев,2021.0,"драмы, военные, приключения",Россия,12.0
19,1097561,14317,0.143034,3,film,Веном,2018.0,"популярное, фантастика, триллеры, боевики, ужасы",США,16.0
9,1097561,3734,0.140239,4,film,Прабабушка легкого поведения,2021.0,комедии,Россия,16.0
20,1097561,7626,0.124257,5,film,Мстители: Война бесконечности,2018.0,"боевики, фантастика, приключения",США,16.0
21,1097561,11237,0.120163,6,film,День города,2021.0,комедии,Россия,16.0
23,1097561,2657,0.116982,7,series,Подслушано,2021.0,"драмы, триллеры",Россия,16.0
24,1097561,12173,0.116811,8,film,Мстители: Финал,2019.0,"боевики, драмы, фантастика",США,16.0
25,1097561,6455,0.091032,9,film,Альфа,2018.0,"драмы, популярное, семейное, приключения",США,12.0
12,1097561,6809,0.087945,10,film,Дуров,2021.0,документальное,Россия,16.0


#### Recos for old man

In [25]:
als_with_features_reco[20:]

,user_id,item_id,score,rank,content_type,title,release_year,genres,countries,age_rating
15,1097563,13865,0.295099,1,film,Девятаев,2021.0,"драмы, военные, приключения",Россия,12.0
4,1097563,9728,0.240718,2,film,Гнев человеческий,2021.0,"боевики, триллеры","Великобритания, США",18.0
7,1097563,10440,0.107948,3,series,Хрустальный,2021.0,"триллеры, детективы",Россия,18.0
10,1097563,3734,0.093120,4,film,Прабабушка легкого поведения,2021.0,комедии,Россия,16.0
26,1097563,4740,0.085816,5,film,Сахаров. Две жизни,2021.0,документальное,Россия,16.0
22,1097563,11237,0.083679,6,film,День города,2021.0,комедии,Россия,16.0
27,1097563,9996,0.074600,7,series,Немцы,2021.0,драмы,Россия,16.0
1,1097563,15297,0.071103,8,series,Клиника счастья,2021.0,"драмы, мелодрамы",Россия,18.0
28,1097563,494,0.067429,9,series,Д'Артаньян и три мушкетера,1979.0,"музыкальные, приключения, исторические, советс...",СССР,0.0
29,1097563,3869,0.064190,10,film,Воздушный извозчик,1943.0,"советские, музыкальные, военные, комедии",СССР,12.0


### ANN

In [26]:
ann_reco = ann.get_item_list_for_user_batch(
    user_ids=artificial_users_ids,
    top_n=K_RECOS
)

In [27]:
ann_recos_df = pd.DataFrame({
    Columns.User: artificial_users_ids,
    'reco': ann_reco
})

In [28]:
ann_recos_df = ann_recos_df\
    .set_index('user_id')\
    .explode('reco')\
    .rename(columns={
        'reco': 'item_id'
    }).reset_index()\
    .merge(items_descriptions, on='item_id')\
    .sort_values(by=['user_id'])

#### Recos for mother with kid

In [29]:
ann_recos_df[:10]

,user_id,item_id,content_type,title,release_year,genres,countries,age_rating
0,1097559,15297,series,Клиника счастья,2021.0,"драмы, мелодрамы",Россия,18.0
1,1097559,9728,film,Гнев человеческий,2021.0,"боевики, триллеры","Великобритания, США",18.0
3,1097559,4151,series,Секреты семейной жизни,2021.0,комедии,Россия,18.0
4,1097559,10440,series,Хрустальный,2021.0,"триллеры, детективы",Россия,18.0
5,1097559,3734,film,Прабабушка легкого поведения,2021.0,комедии,Россия,16.0
7,1097559,6809,film,Дуров,2021.0,документальное,Россия,16.0
8,1097559,7571,film,100% волк,2020.0,"мультфильм, приключения, семейное, фэнтези, ко...","Австралия, Бельгия",6.0
9,1097559,13865,film,Девятаев,2021.0,"драмы, военные, приключения",Россия,12.0
11,1097559,7582,film,Холодное сердце II,2019.0,"фэнтези, мультфильм, музыкальные",США,6.0
12,1097559,16166,film,Зверополис,2016.0,"приключения, мультфильм, детективы, комедии",США,6.0


#### Recos for superheroins fan

In [30]:
ann_recos_df[10:20]

,user_id,item_id,content_type,title,release_year,genres,countries,age_rating
19,1097561,10942,film,Мстители,2012.0,"боевики, фантастика, фэнтези, приключения",США,12.0
18,1097561,12173,film,Мстители: Финал,2019.0,"боевики, драмы, фантастика",США,16.0
17,1097561,2657,series,Подслушано,2021.0,"драмы, триллеры",Россия,16.0
16,1097561,11237,film,День города,2021.0,комедии,Россия,16.0
15,1097561,7626,film,Мстители: Война бесконечности,2018.0,"боевики, фантастика, приключения",США,16.0
14,1097561,6738,film,Человек-паук: Возвращение домой,2017.0,"боевики, популярное, фантастика, приключения",США,16.0
13,1097561,14317,film,Веном,2018.0,"популярное, фантастика, триллеры, боевики, ужасы",США,16.0
10,1097561,13865,film,Девятаев,2021.0,"драмы, военные, приключения",Россия,12.0
6,1097561,3734,film,Прабабушка легкого поведения,2021.0,комедии,Россия,16.0
2,1097561,9728,film,Гнев человеческий,2021.0,"боевики, триллеры","Великобритания, США",18.0


#### Recos for old man

In [31]:
ann_recos_df[20:]

,user_id,item_id,content_type,title,release_year,genres,countries,age_rating
20,1097563,494,series,Д'Артаньян и три мушкетера,1979.0,"музыкальные, приключения, исторические, советс...",СССР,0.0
21,1097563,1012,film,Свадьба в Малиновке,1967.0,"музыкальные, советские, мелодрамы, семейное, к...",СССР,12.0
22,1097563,3869,film,Воздушный извозчик,1943.0,"советские, музыкальные, военные, комедии",СССР,12.0
23,1097563,16394,film,"Не плачь, девчонка",1976.0,"семейное, советские, музыкальные, комедии",СССР,12.0
24,1097563,1538,film,Князь Игорь,1969.0,"музыкальные, драмы, исторические, советские, ф...",СССР,0.0
25,1097563,13588,film,Волшебная сила,1970.0,"советские, семейное, музыкальные, комедии",СССР,0.0
26,1097563,6401,film,Король-олень,1970.0,"музыкальные, советские, семейное, фэнтези, ком...",СССР,0.0
27,1097563,7449,film,Свой парень,1974.0,"мелодрамы, советские, музыкальные, комедии",СССР,16.0
28,1097563,14160,film,Ключи от неба,1965.0,"мелодрамы, советские, музыкальные, комедии",СССР,12.0
29,1097563,15899,film,На войне как на войне,1968.0,"исторические, советские, военные",СССР,12.0


## Conclusion

ALS keeps trying and trying to push popular items into the recommendations. But ANN modification seems to have caught a little more context and gave us more relevant recommendations (although it duplicated some of the aitems)